In [10]:
import requests
import csv
from io import StringIO

rank="A*"
rank="A"
rank="B"
rank="C"

rank_name=rank.replace("*","star")

core_versions = ["CORE2008", "CORE2013", "CORE2014", "CORE2017", "CORE2018", "CORE2020", "CORE2021", "CORE2023"]
url = "https://portal.core.edu.au/conf-ranks/"

for core in core_versions:
    print("▶️ Downloading version: {}".format(core))
    page = 1
    all_rows = []
    header_saved = False

    while True:
        params = {
            "search": rank,
            "by": "rank",
            "source": core,
            "sort": "asource",
            "page": page,
            "do": "Export"
        }

        response = requests.get(url, params=params)
        if response.status_code == 200:
            csv_data = response.content.decode("utf-8")
            csv_reader = csv.reader(StringIO(csv_data))
            rows = list(csv_reader)

            if not rows or len(rows) <= 1:
                break  # No data

            if not header_saved:
                all_rows.append(rows[0])
                header_saved = True

            all_rows.extend(rows[1:])

            if len(rows) < 51:
                break  # Last page
            page += 1
        else:
            print("Could not fetch {} page {}: response was {}".format(core, page, response.status_code))
            break

    filename = "core_{}_{}_export.csv".format(rank_name,core)
    with open(filename, "w", newline='', encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerows(all_rows)
    print("✅ Saved: {}".format(filename))

▶️ Downloading version: CORE2008
❌ Error fetching CORE2008 page 7: 500
✅ Saved: core_C_CORE2008_export.csv
▶️ Downloading version: CORE2013
❌ Error fetching CORE2013 page 18: 500
✅ Saved: core_C_CORE2013_export.csv
▶️ Downloading version: CORE2014
❌ Error fetching CORE2014 page 18: 500
✅ Saved: core_C_CORE2014_export.csv
▶️ Downloading version: CORE2017
❌ Error fetching CORE2017 page 17: 500
✅ Saved: core_C_CORE2017_export.csv
▶️ Downloading version: CORE2018
❌ Error fetching CORE2018 page 17: 500
✅ Saved: core_C_CORE2018_export.csv
▶️ Downloading version: CORE2020
❌ Error fetching CORE2020 page 6: 500
✅ Saved: core_C_CORE2020_export.csv
▶️ Downloading version: CORE2021
❌ Error fetching CORE2021 page 7: 500
✅ Saved: core_C_CORE2021_export.csv
▶️ Downloading version: CORE2023
❌ Error fetching CORE2023 page 9: 500
✅ Saved: core_C_CORE2023_export.csv


In [11]:
import requests
from bs4 import BeautifulSoup
import csv

BASE_URL = "https://portal.core.edu.au/conf-ranks/"
HEADERS = {
    "User-Agent": "Mozilla/5.0"
}

all_rows = []
page = 1

while True:
    print(f"🔄 Feldolgozás: {page}. oldal...")
    params = {
        "search": rank,
        "by": "rank",
        "source": "all",
        "sort": "asource",
        "page": page
    }

    response = requests.get(BASE_URL, params=params, headers=HEADERS)
    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find("table")
    if not table:
        print("🚫 Nem található táblázat ezen az oldalon.")
        break

    rows = table.find_all("tr")[1:]  # Skip header
    if not rows or len(rows) < 50:
        print("✅ Utolsó oldal elérve.")
        break

    for row in rows:
        cols = row.find_all("td")
        if len(cols) != 9:
            continue

        title = cols[0].text.strip()
        acronym = cols[1].text.strip()
        source = cols[2].text.strip()
        rank = cols[3].text.strip()
        note = cols[4].text.strip()
        dblp_tag = cols[5].find("a")
        dblp = dblp_tag["href"].strip() if dblp_tag else ""
        primary_for = cols[6].text.strip()
        comments = cols[7].text.strip()
        avg_rating = cols[8].text.strip()

        all_rows.append([
            title, acronym, source, rank, note,
            dblp, primary_for, comments, avg_rating
        ])

    page += 1

# CSV mentés
filename = "core_{}_with_dblp.csv".format(rank_name)
with open(filename, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow([
        "Title", "Acronym", "Source", "Rank", "Note",
        "DBLP", "Primary FoR", "Comments", "Average Rating"
    ])
    writer.writerows(all_rows)

print("✅ Mentés kész: {}".format(filename))

🔄 Feldolgozás: 1. oldal...
🔄 Feldolgozás: 2. oldal...
🔄 Feldolgozás: 3. oldal...
🔄 Feldolgozás: 4. oldal...
🔄 Feldolgozás: 5. oldal...
🔄 Feldolgozás: 6. oldal...
🔄 Feldolgozás: 7. oldal...
🔄 Feldolgozás: 8. oldal...
🔄 Feldolgozás: 9. oldal...
🔄 Feldolgozás: 10. oldal...
🔄 Feldolgozás: 11. oldal...
🔄 Feldolgozás: 12. oldal...
🔄 Feldolgozás: 13. oldal...
🔄 Feldolgozás: 14. oldal...
🔄 Feldolgozás: 15. oldal...
🔄 Feldolgozás: 16. oldal...
🔄 Feldolgozás: 17. oldal...
🔄 Feldolgozás: 18. oldal...
🔄 Feldolgozás: 19. oldal...
🔄 Feldolgozás: 20. oldal...
🔄 Feldolgozás: 21. oldal...
🔄 Feldolgozás: 22. oldal...
✅ Utolsó oldal elérve.
✅ Mentés kész: core_C_with_dblp.csv


In [12]:
import pandas as pd
import glob
import os
import json

# 📁 Beállítások
directory = "./"
csv_files = glob.glob(os.path.join(directory, "core_{}_*_export.csv".format(rank_name)))
dblp_file = "core_{}_with_dblp.csv".format(rank_name)

# 🔄 Új struktúra: Acronym alapján
acronym_to_info = {}

# 🔄 CSV fájlok feldolgozása
for file in csv_files:
    year = os.path.basename(file).split("_")[2]
    df = pd.read_csv(file)

    df.columns.values[0] = "ID"
    df.columns.values[1] = "Name"
    df.columns.values[2] = "Acronym"

    for _, row in df.iterrows():
        acronym = row["Acronym"]
        if pd.isna(acronym):
            continue  # 🛑 Skip rows with missing acronyms
        if acronym not in acronym_to_info:
            acronym_to_info[acronym] = {
                "Acronym": acronym,
                "Name": row["Name"],
                "YearsListed": [year],
                "DBLP ID": row["ID"]
            }
        else:
            if year not in acronym_to_info[acronym]["YearsListed"]: 
                acronym_to_info[acronym]["YearsListed"].append(year)

# 📥 DBLP fájl beolvasása
dblp_df = pd.read_csv(dblp_file)
if "Acronym" not in dblp_df.columns or "DBLP" not in dblp_df.columns:
    raise KeyError("Acronym vagy DBLP oszlop hiányzik a DBLP fájlból.")

# 🔗 DBLP linkek hozzáadása a dictionary-hez
for _, row in dblp_df.iterrows():
    acronym = row["Acronym"]
    if acronym in acronym_to_info:
        acronym_to_info[acronym]["DBLP URL"] = row["DBLP"]
        acronym_to_info[acronym]["Average Rating"] = str(row.get("Average Rating", None))

# 🧾 Táblázat építése
final_df = pd.DataFrame(acronym_to_info.values())
final_df["YearsListed"] = final_df["YearsListed"].apply(lambda y: ", ".join(sorted(y)))

# 💾 Mentés
final_df.to_csv("core_{}_merged_5cols_by_acronym.csv".format(rank_name), index=False)
print("✅ Elmentve: core_{}_merged_5cols_by_acronym.csv".format(rank_name))

# 📅 Intervallum hozzáadása, 2008 és 2023 elhagyásával
# 📅 Intervallum hozzáadása, 2008 és 2023 elhagyásával
all_years_int = set()
for conf in acronym_to_info.values():
    all_years_int.update(int(y.replace("CORE", "")) for y in conf["YearsListed"])
year_min = min(all_years_int)
year_max = max(all_years_int)

for conf in acronym_to_info.values():
    years = sorted(int(y.replace("CORE", "")) for y in conf["YearsListed"])
    start = min(years)
    end = max(years)

    if start == year_min and end == year_max:
        conf["YearsInterval"] = ""
    elif start == year_min:
        conf["YearsInterval"] = f"–{end}"
    elif end == year_max:
        conf["YearsInterval"] = f"{start}–"
    else:
        conf["YearsInterval"] = f"{start}–{end}"
    

with open("core_{}_conferences.json".format(rank_name), "w", encoding="utf-8") as f:
    json.dump(acronym_to_info, f, indent=2, ensure_ascii=False)
print("✅ Elmentve: core_{}_conferences.json with {} records".format(rank_name, len(acronym_to_info)))

✅ Elmentve: core_C_merged_5cols_by_acronym.csv
✅ Elmentve: core_C_conferences.json with 1004 records


In [ ]:
import json
import os

normalize=0
normalize_all=0
for rank in ["Astar","A","B","C"]:
    try:
        with open(f'core_{rank}_conferences_classified.json', 'r', encoding='utf-8') as f:
            conferences = json.load(f)

        in_last_core = 0
        for acronym, info in conferences.items():
            if "CORE2023" in info.get("YearsListed", []):
                in_last_core += 1
        if normalize==0:
            normalize=in_last_core
        if normalize_all==0:
            normalize_all=len(conferences)
        print(f"🏷️ {rank} konferenciák száma: {len(conferences)} (normalized: {len(conferences)/normalize_all}) az utolsóban {in_last_core} (normalized: {in_last_core/normalize})")

    except Exception as e:
        print(f"🚫 Hiba a {rank} fájl feldolgozásakor: {e}")

        old_name = f"core_{rank}_conferences.json"
        new_name = f"core_{rank}_conferences_classified.json"

        # use notebook-level flags if present
        force = globals().get("force", False)
        dry_run = globals().get("dry_run", False)

        if not os.path.exists(old_name):
            print(f"[skip] {old_name} not found")
            continue

        if os.path.exists(new_name) and not force:
            print(f"[exists] {new_name} already exists (set force=True to overwrite)")
            continue

        print(f"Renaming: {old_name} -> {new_name}")
        if not dry_run:
            if os.path.exists(new_name) and force:
                os.remove(new_name)
            os.rename(old_name, new_name)

🏷️ Astar konferenciák száma: 80 (normalized: 1.0) az utolsóban 59 (nromalized: 1.0)
🏷️ A konferenciák száma: 327 (normalized: 4.0875) az utolsóban 116 (nromalized: 1.9661016949152543)
🏷️ B konferenciák száma: 621 (normalized: 7.7625) az utolsóban 220 (nromalized: 3.7288135593220337)
🏷️ C konferenciák száma: 1004 (normalized: 12.55) az utolsóban 357 (nromalized: 6.0508474576271185)
